In [ ]:
import numpy as np
import os
from utils_mitgcm import open_mitgcm_ds_from_config
from utils_signal_processing import *

In [ ]:
import dask
from dask.distributed import Client
dask.config.set({
    'distributed.worker.memory.target': 0.6,  # fraction of memory to start spilling
    'distributed.worker.memory.spill': 0.7,   # fraction to spill to disk
    'distributed.worker.memory.pause': 0.8,   # fraction to pause worker
})

client = Client(processes=True, n_workers=20, threads_per_worker=1)

In [ ]:
client.dashboard_link

ssh -L 8787:127.0.0.1:8787 leroquan@KBALL-LAKEN-L

In [ ]:
model = 'dummy_rectangle'
mitgcm_config, ds = open_mitgcm_ds_from_config('..//config.json', model)

In [ ]:
folder_path = os.path.dirname(mitgcm_config['datapath'])
output_folder = os.path.join(folder_path, "seiche_analysis")
os.makedirs(output_folder, exist_ok=True)

# Load data

In [ ]:
grid_resolution = 200
ds['YC'] = np.arange(1, len(ds['YC'])+1) * grid_resolution - grid_resolution/2
ds['XC'] = np.arange(1, len(ds['XC'])+1) * grid_resolution - grid_resolution/2
ds['YG'] = np.arange(0, len(ds['YG'])) * grid_resolution
ds['XG'] = np.arange(0, len(ds['XG'])) * grid_resolution

In [ ]:
u = ds.UVEL.chunk({'time':-1,'Z':1})
v = ds.VVEL.chunk({'time':-1,'Z':1})
w = ds.WVEL.chunk({'time':-1,'Zl':1})

# Defining cutoff for filtering

In [ ]:
cutoff1_hr = 14
cutoff2_hr = 8

In [ ]:
path_cutoff_folder = os.path.join(output_folder, f'{cutoff2_hr}_{cutoff1_hr}h')
os.makedirs(path_cutoff_folder, exist_ok=True)

# Filter timeseries

In [ ]:
u_filtered = filter_signal_xarray(u, btype='bandpass', time_dim='time', dt=3600, period_cutoff_low=(cutoff1_hr*3600), period_cutoff_high=(cutoff2_hr*3600), order=5)
v_filtered = filter_signal_xarray(v, btype='bandpass', time_dim='time', dt=3600, period_cutoff_low=(cutoff1_hr*3600), period_cutoff_high=(cutoff2_hr*3600), order=5)
w_filtered = filter_signal_xarray(w, btype='bandpass', time_dim='time', dt=3600, period_cutoff_low=(cutoff1_hr*3600), period_cutoff_high=(cutoff2_hr*3600), order=5)

In [ ]:
u_filtered.isel(Z=5,YC=5,XG=5).plot()

# Save results

In [ ]:
u_filtered['drF'] = u_filtered['drF'].astype('<f8')
v_filtered['drF'] = v_filtered['drF'].astype('<f8')

In [ ]:
coords_to_drop = ["dyG", "dxC", "dxG", "dyC", "rAs", "hFacS", "rAw", "PHrefC", "hFacW", "rhoRef", "rA", "Depth", "dxF", "dyF"]

In [ ]:
u_filtered.drop_vars(coords_to_drop, errors='ignore').to_zarr(os.path.join(path_cutoff_folder, rf"u_filtered_{cutoff2_hr}-{cutoff1_hr}h.zarr"), mode='w')
v_filtered.drop_vars(coords_to_drop, errors='ignore').to_zarr(os.path.join(path_cutoff_folder, rf"v_filtered_{cutoff2_hr}-{cutoff1_hr}h.zarr"), mode='w')
w_filtered.drop_vars(coords_to_drop, errors='ignore').to_zarr(os.path.join(path_cutoff_folder, rf"w_filtered_{cutoff2_hr}-{cutoff1_hr}h.zarr"), mode='w')

# Save residual

In [ ]:
u_residual = (u - u_filtered)
v_residual = (v - v_filtered)
w_residual = (w - w_filtered)

In [ ]:
u_residual['drF'] = u_residual['drF'].astype('<f8')
v_residual['drF'] = v_residual['drF'].astype('<f8')

u_residual.drop_vars(coords_to_drop, errors='ignore').to_zarr(os.path.join(path_cutoff_folder, rf"u_residual_{cutoff2_hr}-{cutoff1_hr}h.zarr"), mode='w')
v_residual.drop_vars(coords_to_drop, errors='ignore').to_zarr(os.path.join(path_cutoff_folder, rf"v_residual_{cutoff2_hr}-{cutoff1_hr}h.zarr"), mode='w')
w_residual.drop_vars(coords_to_drop, errors='ignore').to_zarr(os.path.join(path_cutoff_folder, rf"w_residual_{cutoff2_hr}-{cutoff1_hr}h.zarr"), mode='w')

In [ ]:
import matplotlib.pyplot as plt
(u_filtered**2+u_residual**2).isel(Z=20,YC=5,XG=18).plot(label='filtered')
(u**2).isel(Z=20,YC=5,XG=18).plot(label='total')
plt.legend()